In [1]:
# Test for Pytorch and CUDA version
import torch; print(torch.__version__)
print(torch.version.cuda)

1.12.1
11.6


In [2]:
from Dataset_Loader_Node_Classification import Dataset_Loader
# import torch
import torch.nn as nn
import torch_geometric.nn as geonn
import numpy as np

In [5]:
obj = Dataset_Loader(dDataset='cora')
obj.dataset_source_folder_path='data/cora/'
data = obj.load()
# Try print some test data

print(data['train_test']['idx_train'])
print(f"The value of graph: {data['graph']['node']}")

Loading cora dataset...
range(0, 200)
The value of graph: {31336: 0, 1061127: 1, 1106406: 2, 13195: 3, 37879: 4, 1126012: 5, 1107140: 6, 1102850: 7, 31349: 8, 1106418: 9, 1123188: 10, 1128990: 11, 109323: 12, 217139: 13, 31353: 14, 32083: 15, 1126029: 16, 1118017: 17, 49482: 18, 753265: 19, 249858: 20, 1113739: 21, 48766: 22, 646195: 23, 1126050: 24, 59626: 25, 340299: 26, 354004: 27, 242637: 28, 1106492: 29, 74975: 30, 1152272: 31, 100701: 32, 66982: 33, 13960: 34, 13966: 35, 66990: 36, 182093: 37, 182094: 38, 13972: 39, 13982: 40, 16819: 41, 273152: 42, 237521: 43, 1153703: 44, 32872: 45, 284025: 46, 218666: 47, 16843: 48, 1153724: 49, 1153728: 50, 158098: 51, 8699: 52, 1134865: 53, 28456: 54, 248425: 55, 1112319: 56, 28471: 57, 175548: 58, 696345: 59, 28485: 60, 1139195: 61, 35778: 62, 28491: 63, 310530: 64, 1153784: 65, 1481: 66, 1153786: 67, 13212: 68, 1111614: 69, 5055: 70, 4329: 71, 330148: 72, 1105062: 73, 4330: 74, 5062: 75, 4335: 76, 158812: 77, 40124: 78, 1103610: 79, 688361

In [6]:
# Print the dimension of features input
feature = data['graph']['X']
print(feature.shape)
print(f'The first features: {feature[0]}')


(2708, 1433)
The first features: [[0. 0. 0. ... 0. 0. 0.]]


In [8]:
# Print dimension of classification output
labels = torch.tensor(data['graph']['y'])
print(labels[0])
print(f'Number of unique values in: {labels.unique()}')
labels.shape

tensor(6)
Number of unique values in: tensor([0, 1, 2, 3, 4, 5, 6])


torch.Size([2708])

In [9]:
# Check the edge index tensor shape
edges = data['graph']['edge']
print(edges[0])
# So 163 402 is the index of the edge connection, now lest check if it's 35 1033
print('The actual id getting from index is')
print(f"{data['graph']['utility']['reverse_idx'][edges[0][0]]} and {data['graph']['utility']['reverse_idx'][edges[0][1]]}")

[163 402]
The actual id getting from index is
35 and 1033


In [10]:
# Great, now we can get all the edges that matter in our training set
idxTrain = data['train_test']['idx_train']
edgeMatterList = []
for idx in idxTrain:
    linksWithThisIdx = [e.tolist() for e in edges if e[0] < 200 and e[1] < 200] # sort out all the index that is not in
    if linksWithThisIdx:
        edgeMatterList.extend(linksWithThisIdx)
print(edgeMatterList)
edgeMatterTensor = torch.tensor(edgeMatterList)
print(edgeMatterTensor)

[[163, 141], [163, 174], [163, 191], [163, 188], [163, 55], [163, 42], [163, 145], [163, 129], [163, 22], [87, 47], [66, 69], [66, 183], [66, 34], [66, 149], [74, 154], [74, 71], [74, 19], [76, 131], [76, 181], [84, 12], [84, 155], [84, 27], [122, 124], [122, 127], [127, 124], [136, 92], [136, 159], [158, 184], [192, 10], [192, 193], [193, 10], [193, 74], [3, 197], [35, 34], [39, 24], [39, 82], [109, 171], [114, 171], [41, 94], [41, 23], [41, 115], [157, 181], [157, 78], [160, 125], [63, 153], [162, 173], [0, 8], [14, 31], [14, 0], [14, 8], [15, 152], [15, 129], [15, 80], [129, 188], [129, 22], [4, 170], [88, 116], [88, 44], [144, 188], [18, 127], [107, 108], [25, 20], [12, 155], [137, 138], [138, 137], [138, 115], [191, 22], [155, 117], [161, 95], [146, 17], [150, 50], [188, 141], [188, 42], [188, 163], [188, 139], [188, 120], [189, 188], [189, 42], [189, 163], [143, 180], [196, 195], [42, 188], [128, 103], [126, 94], [163, 141], [163, 174], [163, 191], [163, 188], [163, 55], [163, 42

In [11]:
# Check if the connection we load in its correct
print('The actual id getting from index is')
print(f"{data['graph']['utility']['reverse_idx'][edgeMatterList[0][0]]} and {data['graph']['utility']['reverse_idx'][edgeMatterList[0][1]]}")

The actual id getting from index is
35 and 1117476


In [12]:
# ! EDGE matter
inputEdge = edgeMatterTensor.T
print(inputEdge)
inputEdge.shape

tensor([[163, 163, 163,  ...,  42, 128, 126],
        [141, 174, 191,  ..., 188, 103,  94]])


torch.Size([2, 16800])

In [14]:
torch.tensor(idxTrain[0]).item()

0

In [16]:
# See if the training index is the same
print(f"{data['graph']['utility']['reverse_idx'][idxTrain[0]]}")

31336


In [17]:
# Check the shape of the total edge index
data['graph']['edge'].shape

(5429, 2)

In [18]:
# ! Rethink about using EDGE AGAIN
print(edges)
reShapeEdge = edges.T
reShapeEdge


[[ 163  402]
 [ 163  659]
 [ 163 1696]
 ...
 [1887 2258]
 [1902 1887]
 [ 837 1686]]


array([[ 163,  163,  163, ..., 1887, 1902,  837],
       [ 402,  659, 1696, ..., 2258, 1887, 1686]], dtype=int32)

In [19]:
# What does the adj matrix looks like?
# It doesn't matter, for other stuff
adjMatrix = data['graph']['utility']['A']
print(adjMatrix.shape)
print(adjMatrix.coalesce().indices())

torch.Size([2708, 2708])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    8,   14,  ..., 1389, 2344, 2707]])


In [20]:
# The adj matrix is the edge index that has been symetric normalized
# Have to convert it to shape of [2, edge_numbers] to pass it in
edgeIndexInput = adjMatrix.coalesce().indices()
print(edgeIndexInput.shape)

print(edgeIndexInput)

torch.Size([2, 13264])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    8,   14,  ..., 1389, 2344, 2707]])


In [21]:
workingEdgeIndex = edgeIndexInput.numpy()
print(np.where(workingEdgeIndex[0] >= 200))
print(workingEdgeIndex[0][1260:1269])
workingEdgeIndex[1][1260:1269]

(array([ 1268,  1269,  1270, ..., 13261, 13262, 13263]),)
[197 197 198 198 198 199 199 199 200]


array([633, 749, 198, 655, 747, 199, 591, 640, 200])

In [22]:
# What's the actual node id of those index
print('The actual id getting from index is')
print(f"{data['graph']['utility']['reverse_idx'][0]} and connect from {data['graph']['utility']['reverse_idx'][8]}")
print(f"{data['graph']['utility']['reverse_idx'][0]} and connect from {data['graph']['utility']['reverse_idx'][14]}")
print(f"{data['graph']['utility']['reverse_idx'][199]} and connect from {data['graph']['utility']['reverse_idx'][640]}")

The actual id getting from index is
31336 and connect from 31349
31336 and connect from 31353
68505 and connect from 10177


In [23]:
# The data seems to in in transposed? Also a few indices seems to point to itself?

# Check pointing to itself
pointItself = []
for i in range(13264):
    if workingEdgeIndex[0][i] == workingEdgeIndex[1][i]:
        print(workingEdgeIndex[0][i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
# Build the X and y through index
X = feature[data['train_test']['idx_train']]
y = labels[data['train_test']['idx_train']]

In [25]:
print(X.shape)
print(y.shape)
print(edgeIndexInput.shape)
    # Need to lower the size of edgeIndexInput also if want to do smaller X size

(200, 1433)
torch.Size([200])
torch.Size([2, 13264])


In [26]:
print(data['train_test']['idx_train'])


range(0, 200)


In [27]:
# Great, now we can pass them through model
# ? https://github.com/pyg-team/pytorch_geometric
torch.autograd.anomaly_mode.set_detect_anomaly(True)
model = geonn.Sequential('x, edge_idx',[
    (geonn.GCNConv(in_channels=1433, out_channels=50), 'x, edge_idx -> x'),
    nn.ReLU(),
    (geonn.GCNConv(in_channels=50, out_channels=7), 'x, edge_idx -> x'),
])

'''
Note that edge_index, i.e. the tensor defining the source and target 
nodes of all edges, is not a list of index tuples. If you want to 
write your indices this way, you should transpose and call contiguous 
on it before passing them to the data constructor:
'''

res = model(x=X, edge_idx=inputEdge)

# OK we use EDGE THAT MATTER

TypeError: 'int' object is not callable

In [ ]:
print(res)

tensor([[ 2.4122e+00,  3.8015e-01, -3.2871e+00,  ...,  1.1502e+00,
          7.9738e-01,  2.5619e-01],
        [ 1.3918e-01,  8.3823e-02, -9.7496e-03,  ..., -5.4123e-02,
         -1.8349e-02,  3.0424e-02],
        [ 3.2005e-01,  1.5628e-01,  4.7347e-02,  ...,  3.2707e-03,
         -1.1811e-01,  5.4228e-02],
        ...,
        [ 4.7425e+00, -6.1567e-01, -7.4744e-01,  ..., -1.5278e-01,
          2.4097e+00,  3.0896e-01],
        [ 4.8336e-02,  2.7772e-02,  1.7923e-01,  ..., -3.1802e-02,
          8.7977e-02,  9.4847e-02],
        [ 5.1331e-02,  9.0444e-02,  3.7649e-02,  ...,  5.0408e-03,
          4.1575e-02,  5.8994e-03]], grad_fn=<AddBackward0>)


In [ ]:
# Lest try the prototype first
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='.', name='Cora')

In [ ]:
anotherData = dataset[0]
print(anotherData.edge_index)
print(anotherData.edge_index.shape)

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
torch.Size([2, 10556])


In [ ]:
print(anotherData.x)
print(anotherData.x.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([2708, 1433])
